# ETH HMA 数据分析 Notebook

这个Notebook用于分析ETH历史数据和HMA技术指标，帮助您进行量化分析和策略开发。

## 数据概览
- **数据源**: 币安API
- **时间范围**: 2022年9月 - 2025年9月
- **时间间隔**: 1小时和4小时
- **技术指标**: HMA_45, SMA_20, SMA_50等


## 1. 导入必要的库


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 设置pandas显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("✅ 库导入完成")


## 2. 加载数据


In [ ]:
# 加载1小时和4小时的处理后数据
data_dir = Path('data')

# 自动查找最新的处理文件
processed_files = list(data_dir.glob("ETHUSDT_*_processed_*.parquet"))

# 加载数据
df_1h = None
df_4h = None

for file_path in processed_files:
    if '1h' in file_path.name:
        df_1h = pd.read_parquet(file_path)
        df_1h.set_index('open_time', inplace=True)
        print(f"✅ 1小时数据加载完成: {len(df_1h):,} 条记录")
    elif '4h' in file_path.name:
        df_4h = pd.read_parquet(file_path)
        df_4h.set_index('open_time', inplace=True)
        print(f"✅ 4小时数据加载完成: {len(df_4h):,} 条记录")

if df_1h is not None:
    print(f"📅 时间范围: {df_1h.index.min()} 到 {df_1h.index.max()}")
    print(f"📊 数据列: {list(df_1h.columns)}")


## 3. 数据概览和基本信息


In [ ]:
# 显示数据基本信息
print("📊 1小时数据概览:")
print(f"数据形状: {df_1h.shape}")
print(f"列名: {list(df_1h.columns)}")
print("\n前5行数据:")
df_1h.head()


In [ ]:
# 显示数据统计信息
print("📈 价格统计信息 (1小时数据):")
price_stats = df_1h[['open', 'high', 'low', 'close', 'volume']].describe()
print(price_stats)

print("\n🔍 HMA指标统计信息:")
hma_stats = df_1h[['HMA_45', 'price_change', 'hma_deviation']].describe()
print(hma_stats)


## 4. 价格走势可视化


In [ ]:
# 创建价格走势图
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('ETH HMA 分析', fontsize=16, fontweight='bold')

# 1. 收盘价走势
axes[0, 0].plot(df_1h.index, df_1h['close'], linewidth=0.8, alpha=0.8)
axes[0, 0].set_title('ETH 收盘价走势 (1小时)')
axes[0, 0].set_ylabel('价格 (USDT)')
axes[0, 0].grid(True, alpha=0.3)

# 2. 价格与HMA对比
axes[0, 1].plot(df_1h.index, df_1h['close'], label='收盘价', linewidth=0.8, alpha=0.8)
axes[0, 1].plot(df_1h.index, df_1h['HMA_45'], label='HMA_45', linewidth=1.2, alpha=0.9)
axes[0, 1].set_title('价格与HMA对比')
axes[0, 1].set_ylabel('价格 (USDT)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. 成交量
axes[1, 0].plot(df_1h.index, df_1h['volume'], linewidth=0.8, alpha=0.8, color='orange')
axes[1, 0].set_title('成交量走势')
axes[1, 0].set_ylabel('成交量 (ETH)')
axes[1, 0].grid(True, alpha=0.3)

# 4. HMA偏离度
axes[1, 1].plot(df_1h.index, df_1h['hma_deviation'], linewidth=0.8, alpha=0.8, color='purple')
axes[1, 1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1, 1].set_title('HMA与价格偏离度')
axes[1, 1].set_ylabel('偏离度 (%)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
